In [1]:
!pip install selenium
!pip install webdriver-manager

In [2]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [C:\Users\beomj\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache
 


In [3]:
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions

addr = "https://tv.naver.com/jtbc.skycastle/talks" 
# 크롤링하고자하는 사이트 주소
# 개인적으론 모바일 페이지로 하는게 더 가볍고 빠를것같은 기분이 든다.

driver.get(addr)
time.sleep(1.5)
comments = []
date = []
cur_page = []   


for P in range(1000):
    try:
        if P!=0:
            driver.find_element_by_link_text("다음").click()
            time.sleep(1)
        html = driver.page_source
        dom = BeautifulSoup(html, "lxml")
        comments_raw = dom.find_all("div", {"class" : "u_cbox_area"})
        for comment in comments_raw:    
            comments.append(comment.find("span", {'class': "u_cbox_contents", "data-lang":"ko"}).text)
            date.append(comment.find("span", {'class':'u_cbox_date'}).get('data-value'))
            cur_page.append(P*5+1)
    except:
        pass
    for page in range(2,6):
        try:
            driver.find_element_by_link_text(str(5*P+page)).click()
            time.sleep(1)
            html = driver.page_source
            dom = BeautifulSoup(html, "lxml")

            comments_raw = dom.find_all("div", {"class" : "u_cbox_area"})
            for comment in comments_raw:
                if comment:
                    comments.append(comment.find("span", {'class': "u_cbox_contents", "data-lang":"ko"}).text)
                    date.append(comment.find("span", {'class':'u_cbox_date'}).get('data-value'))
                    cur_page.append(P*5+page)
        except:
                pass
                    
        

In [4]:
df = pd.DataFrame({'comment':comments,'date':date,'page':cur_page})
temp = df.groupby(['comment']).count()

temp.sort_values(by='date',ascending=False)

,date,page
comment,,
ㅋㅋ,9,9
스캐2,8,8
스캐2 ♡♡♡,6,6
ㅋㅋㅋ,5,5
ㅋ,4,4
...,...,...
막화 제일 별로임,1,1
막화 때문에 캔슬,1,1
막판에 작가바뀜?,1,1


In [5]:
# 저장
df.to_csv("df.csv", index=False)